# Baseline with mse loss with wide cnn modified + adam
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=3,4
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=3,4


Using TensorFlow backend.


In [2]:
window_size = 5001
process_all = False
sample_num = 1000

In [3]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [4]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [5]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [6]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [7]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [8]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 5001
[[123409527, 123414528], [123409441, 123414442], [131906064, 131911065]]


In [9]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 5001, 5)


In [10]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [11]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 5001, 5)


In [12]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 5001)


In [13]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 5001)


In [14]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 5001, 1)


In [15]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.97687  0.97687  0.97687  0.97687  0.97687]


In [16]:
from keras.layers import Input, Dense, Conv1D, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [17]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 32
# parameters for training
batch_size = 128
num_epochs = 300
evaluation_freq = 10

In [18]:
'''
Baseline CNN (based on KERAS functional API)
'''
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1)(inputs)
x = Dropout(dropout_rate)(x)
# x = BatchNormalization(momentum=0.8)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [19]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5001, 5)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5001, 32)          800192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5001, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5001, 1)           1025      
Total params: 801,217
Trainable params: 801,217
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [21]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [22]:
model_dir = os.path.join("models", "cnn_5000")
log_dir = os.path.join("logs", "cnn_5000")
#srv_dir = os.path.join("/users", "jesikmin", "mse_lowlr_window4000")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "cnn_5000")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [23]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(lr=1e-3, clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [24]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [25]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.96,
                              patience=5,
                              min_lr=1e-7)

In [26]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [27]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(7500, window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:7500], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(2500, window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[7500:10000], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(1000, window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[10000:11000], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [28]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [29]:
print "Fitting the model..."
X_train, y_train = atac_seq_day0[:7500], histone_mark_day0[:7500]
X_val, y_val = atac_seq_day0[7500:10000], histone_mark_day0[7500:10000]
X_test, y_test = atac_seq_day0[10000:11000], histone_mark_day0[10000:11000]

model.fit(atac_seq_day0[:10000],
          histone_mark_day0[:10000],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.25,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 7500 samples, validate on 2500 samples
Epoch 1/300
7424/7500 [============================>.] - ETA: 0s - loss: 247.5970 - pearson: 0.1557 - mean_squared_error: 247.5970 - mean_absolute_error: 5.7415
Epoch 00001: val_loss improved from inf to 97.93046, saving model to models/cnn_5000/best_model.h5
Writing 1D track of shape: (7500, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 5001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_5000/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============================] - 468s 62ms/step - loss: 246.8408 - pearson: 0.1555 - mean_squared_error: 246.8408 - mean_absolute_error: 5.7391 - val_loss: 97.9305 - val_pearson: 0.1700 - val_mean_squared_e

Epoch 13/300
7424/7500 [============================>.] - ETA: 0s - loss: 214.9570 - pearson: 0.2368 - mean_squared_error: 214.9570 - mean_absolute_error: 5.3482
Epoch 00013: val_loss did not improve
7500/7500 [==============================] - 79s 11ms/step - loss: 214.6071 - pearson: 0.2373 - mean_squared_error: 214.6071 - mean_absolute_error: 5.3495 - val_loss: 95.0014 - val_pearson: 0.1856 - val_mean_squared_error: 95.0014 - val_mean_absolute_error: 2.7689
Epoch 14/300
7424/7500 [============================>.] - ETA: 0s - loss: 212.8789 - pearson: 0.2380 - mean_squared_error: 212.8789 - mean_absolute_error: 5.2708
Epoch 00014: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 213.7158 - pearson: 0.2386 - mean_squared_error: 213.7158 - mean_absolute_error: 5.2826 - val_loss: 94.4489 - val_pearson: 0.1904 - val_mean_squared_error: 94.4489 - val_mean_absolute_error: 3.2181
Epoch 15/300
7424/7500 [============================>.] - ETA: 0s - lo

Epoch 25/300
7424/7500 [============================>.] - ETA: 0s - loss: 208.8630 - pearson: 0.2543 - mean_squared_error: 208.8630 - mean_absolute_error: 5.3731
Epoch 00025: val_loss did not improve
7500/7500 [==============================] - 77s 10ms/step - loss: 209.5352 - pearson: 0.2536 - mean_squared_error: 209.5352 - mean_absolute_error: 5.3795 - val_loss: 92.8145 - val_pearson: 0.1941 - val_mean_squared_error: 92.8145 - val_mean_absolute_error: 2.9540
Epoch 26/300
7424/7500 [============================>.] - ETA: 0s - loss: 207.4383 - pearson: 0.2579 - mean_squared_error: 207.4383 - mean_absolute_error: 5.2752
Epoch 00026: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 207.2714 - pearson: 0.2571 - mean_squared_error: 207.2714 - mean_absolute_error: 5.2812 - val_loss: 92.9006 - val_pearson: 0.1995 - val_mean_squared_error: 92.9006 - val_mean_absolute_error: 2.9506
Epoch 27/300
7424/7500 [============================>.] - ETA: 0s - lo

1000/1000 [==============================] - 4s 4ms/steposs: 204.4116 - pearson: 0.273
[('loss', 117.10597448730469), ('pearson', 0.27635381710529328), ('mean_squared_error', 117.10597442626953), ('mean_absolute_error', 3.2385417947769164)]
Test spearman: 0.290393408341

Epoch 00040: val_loss did not improve
7500/7500 [==============================] - 87s 12ms/step - loss: 204.4443 - pearson: 0.2739 - mean_squared_error: 204.4443 - mean_absolute_error: 5.3583 - val_loss: 94.3286 - val_pearson: 0.1988 - val_mean_squared_error: 94.3286 - val_mean_absolute_error: 2.5556
Epoch 41/300
7424/7500 [============================>.] - ETA: 0s - loss: 206.2906 - pearson: 0.2695 - mean_squared_error: 206.2906 - mean_absolute_error: 5.2751
Epoch 00041: val_loss did not improve
7500/7500 [==============================] - 79s 10ms/step - loss: 205.1632 - pearson: 0.2696 - mean_squared_error: 205.1632 - mean_absolute_error: 5.2648 - val_loss: 92.5327 - val_pearson: 0.1997 - val_mean_squared_error: 92

Epoch 55/300
7424/7500 [============================>.] - ETA: 0s - loss: 199.8940 - pearson: 0.2765 - mean_squared_error: 199.8940 - mean_absolute_error: 5.2711
Epoch 00055: val_loss did not improve
7500/7500 [==============================] - 77s 10ms/step - loss: 202.0948 - pearson: 0.2777 - mean_squared_error: 202.0948 - mean_absolute_error: 5.2928 - val_loss: 92.8213 - val_pearson: 0.1975 - val_mean_squared_error: 92.8213 - val_mean_absolute_error: 2.7926
Epoch 56/300
7424/7500 [============================>.] - ETA: 0s - loss: 202.0195 - pearson: 0.2829 - mean_squared_error: 202.0195 - mean_absolute_error: 5.3646
Epoch 00056: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 201.5050 - pearson: 0.2828 - mean_squared_error: 201.5050 - mean_absolute_error: 5.3599 - val_loss: 92.4126 - val_pearson: 0.1976 - val_mean_squared_error: 92.4126 - val_mean_absolute_error: 2.8286
Epoch 57/300
7424/7500 [============================>.] - ETA: 0s - lo

...

7424/7500 [============================>.] - ETA: 0s - loss: 188.2924 - pearson: 0.3254 - mean_squared_error: 188.2924 - mean_absolute_error: 5.1864
Epoch 00224: val_loss did not improve
7500/7500 [==============================] - 79s 11ms/step - loss: 187.8444 - pearson: 0.3245 - mean_squared_error: 187.8444 - mean_absolute_error: 5.1809 - val_loss: 92.8022 - val_pearson: 0.1918 - val_mean_squared_error: 92.8022 - val_mean_absolute_error: 2.8700
Epoch 225/300
7424/7500 [============================>.] - ETA: 0s - loss: 189.1248 - pearson: 0.3231 - mean_squared_error: 189.1248 - mean_absolute_error: 5.2398
Epoch 00225: val_loss did not improve
7500/7500 [==============================] - 79s 10ms/step - loss: 188.2239 - pearson: 0.3226 - mean_squared_error: 188.2239 - mean_absolute_error: 5.2346 - val_loss: 92.5662 - val_pearson: 0.1955 - val_mean_squared_error: 92.5662 - val_mean_absolute_error: 2.9828
Epoch 226/300
7424/7500 [============================>.] - ETA: 0s - loss: 186.904

7424/7500 [============================>.] - ETA: 0s - loss: 187.2221 - pearson: 0.3274 - mean_squared_error: 187.2221 - mean_absolute_error: 5.2019
Epoch 00241: val_loss did not improve
7500/7500 [==============================] - 78s 10ms/step - loss: 187.3280 - pearson: 0.3280 - mean_squared_error: 187.3280 - mean_absolute_error: 5.1999 - val_loss: 93.6136 - val_pearson: 0.1865 - val_mean_squared_error: 93.6136 - val_mean_absolute_error: 2.8043
Epoch 242/300
7424/7500 [============================>.] - ETA: 0s - loss: 187.7524 - pearson: 0.3342 - mean_squared_error: 187.7524 - mean_absolute_error: 5.1229
Epoch 00242: val_loss did not improve
7500/7500 [==============================] - 79s 10ms/step - loss: 187.7486 - pearson: 0.3323 - mean_squared_error: 187.7486 - mean_absolute_error: 5.1322 - val_loss: 92.9367 - val_pearson: 0.1941 - val_mean_squared_error: 92.9367 - val_mean_absolute_error: 3.0555
Epoch 243/300
7424/7500 [============================>.] - ETA: 0s - loss: 186.828

7424/7500 [============================>.] - ETA: 0s - loss: 187.7521 - pearson: 0.3244 - mean_squared_error: 187.7521 - mean_absolute_error: 5.2052
Epoch 00258: val_loss did not improve
7500/7500 [==============================] - 79s 11ms/step - loss: 187.4794 - pearson: 0.3231 - mean_squared_error: 187.4794 - mean_absolute_error: 5.2001 - val_loss: 92.5585 - val_pearson: 0.1931 - val_mean_squared_error: 92.5585 - val_mean_absolute_error: 2.9439
Epoch 259/300
7424/7500 [============================>.] - ETA: 0s - loss: 183.8573 - pearson: 0.3278 - mean_squared_error: 183.8573 - mean_absolute_error: 5.0894
Epoch 00259: val_loss did not improve
7500/7500 [==============================] - 79s 11ms/step - loss: 186.8375 - pearson: 0.3283 - mean_squared_error: 186.8375 - mean_absolute_error: 5.1136 - val_loss: 92.7527 - val_pearson: 0.1917 - val_mean_squared_error: 92.7527 - val_mean_absolute_error: 2.9120
Epoch 260/300
1000/1000 [==============================] - 4s 4ms/steposs: 186.107

7424/7500 [============================>.] - ETA: 0s - loss: 187.3033 - pearson: 0.3255 - mean_squared_error: 187.3033 - mean_absolute_error: 5.1124
Epoch 00275: val_loss did not improve
7500/7500 [==============================] - 78s 10ms/step - loss: 186.7281 - pearson: 0.3245 - mean_squared_error: 186.7281 - mean_absolute_error: 5.1051 - val_loss: 92.9608 - val_pearson: 0.1901 - val_mean_squared_error: 92.9608 - val_mean_absolute_error: 2.8962
Epoch 276/300
7424/7500 [============================>.] - ETA: 0s - loss: 187.5947 - pearson: 0.3301 - mean_squared_error: 187.5947 - mean_absolute_error: 5.1946
Epoch 00276: val_loss did not improve
7500/7500 [==============================] - 79s 10ms/step - loss: 186.6657 - pearson: 0.3334 - mean_squared_error: 186.6657 - mean_absolute_error: 5.1829 - val_loss: 92.8084 - val_pearson: 0.1931 - val_mean_squared_error: 92.8084 - val_mean_absolute_error: 2.9801
Epoch 277/300
7424/7500 [============================>.] - ETA: 0s - loss: 187.692

7424/7500 [============================>.] - ETA: 0s - loss: 186.5283 - pearson: 0.3306 - mean_squared_error: 186.5283 - mean_absolute_error: 5.1687
Epoch 00292: val_loss did not improve
7500/7500 [==============================] - 79s 11ms/step - loss: 186.5751 - pearson: 0.3292 - mean_squared_error: 186.5751 - mean_absolute_error: 5.1690 - val_loss: 93.3138 - val_pearson: 0.1879 - val_mean_squared_error: 93.3138 - val_mean_absolute_error: 2.9122
Epoch 293/300
7424/7500 [============================>.] - ETA: 0s - loss: 184.8406 - pearson: 0.3320 - mean_squared_error: 184.8406 - mean_absolute_error: 5.1380
Epoch 00293: val_loss did not improve
7500/7500 [==============================] - 79s 11ms/step - loss: 186.2185 - pearson: 0.3319 - mean_squared_error: 186.2185 - mean_absolute_error: 5.1520 - val_loss: 93.0785 - val_pearson: 0.1912 - val_mean_squared_error: 93.0785 - val_mean_absolute_error: 2.9250
Epoch 294/300
7424/7500 [============================>.] - ETA: 0s - loss: 187.011